In [ ]:
80 >> 8080 >>5000

In [ ]:
$ ansible-galaxy collection install amazon.aws

$ ansible-galaxy collection install community.docker



In [ ]:
$ cat hosts 
ansible_host=localhost ansible_connection=local ansible_python_interpreter=/usr/bin/python3

[image-server]
172.31.13.150 ansible_user="ec2-user" ansible_port=22 ansible_ssh_private_key_file="aws.pem" ansible_python_interpreter=/usr/bin/python3
[ec2-user@ip-172-31-5-232 docker-rolling]$ cat aws.vars 
---
access_key: "AKIAXQMPZJZXsadaEAODMSWH"
secret_key: "Ig52kp3QGcBmv7zgypBdafzHLH6TL4ZRRFU3+KYx/XM"
asg_name: "docker-asg"
region_name: "ap-south-1"
[ec2-user@ip-172-31-5-232 docker-rolling]$ cat image.vars 
---
git_url: "https://github.com/jinumona/devops-flask.git"
git_dir: "/var/www/flask-app"
hub_pwd: "xxxxxxxx"
hub_username: "xxxxxx"


### docker-image-git-flask-asg-rolling.yml

In [ ]:
$ cat docker-image-git-flask-asg-rolling.yml 
---
- name: "Creating Docker Image from Git"
  hosts: image-server
  become: yes
  vars_files:
    - image.vars
  tasks:
    - name: "Installing yum packages"
      yum:
        name:
          - git
          - docker 
        state: present
      register: pkg_status

    - name: "restart/enable docker service"
      when: pkg_status.changed == true
      service:
        name: docker 
        state: restarted
        enabled: yes

    - name: "Installing python docker connector module"
      pip:
        executable: pip3
        name:
          - docker
        
    - name: "Appending ec2-user to docker group"
      user:
        append: yes
        groups: docker
        name: ec2-user

    - name: "cloning git dockerfiles"
      git:
        dest: "{{ git_dir }}"
        repo: "{{ git_url }}"
      register: git_status

    - name: "Git fact gathering"
      set_fact:
        cacheable: yes
        git_status: "{{ git_status }}"

    - name: "Logging to DockerHub"
      when: git_status.changed == true
      community.docker.docker_login:
        password: "{{ hub_pwd }}"
        state: present
        username: "{{ hub_username }}" 


    - name: "Building & Pushing Docker latest Image to Hub based on Git version {{ git_status.after }}"
      when: git_status.changed == true
      community.docker.docker_image:
        name: "jinumona/devops-flask:{{ item }}"
        build:
          path: "{{ git_dir }}"
        source: build 
        push: yes
        state: present
        force_source: yes
        force_tag: yes
      with_items:
        - "{{ git_status.after }}"
        -  latest
      register: image_build_status
      


    - name: "log out from DockerHub"
      when: git_status.changed == true
      community.docker.docker_login:
        state: absent

    - name: "Removing created local images"
      when: git_status.changed == true
      community.docker.docker_image:
        name: "jinumona/devops-flask:{{ item }}"
        state: absent
      with_items:
        - "{{ git_status.after }}"
        -  latest

        

- name: "Creating Dynamic hosts Inventry"
  hosts: localhost
  become: yes
  vars_files:
    - aws.vars

  tasks:
    - name: "Installing python aws connector modules"
      pip:
        executable: pip3
        name:
          - boto3 
          - botocore


    - name: "Fetching public ip of instances under Autoscaling Group {{ asg_name }}"
      amazon.aws.ec2_instance_info:
        aws_access_key: "{{ access_key }}"
        aws_secret_key: "{{ secret_key }}"
        region: "{{ region_name }}"
        filters:
          instance-state-name: [ "running"]
          "tag:aws:autoscaling:groupName": "{{ asg_name }}"
      register: asg_instance_info

    - name: "Creating hosts inventry"
      add_host:
        groups: lb_backends
        name: "{{ item.public_ip_address }}"
        ansible_ssh_host: "{{ item.public_ip_address }}"
        ansible_ssh_port: 22
        ansible_ssh_user: "ec2-user"
        ansible_ssh_private_key_file: "aws.pem"
        ansible_ssh_common_args: "-o StrictHostKeyChecking=no"
        ansible_python_interpreter: "/usr/bin/python3"
      with_items: "{{ asg_instance_info.instances }}"

    
- name: "Creating docker containers from latest Image to the backends of ASG "
  hosts: lb_backends
  become: yes
  vars:
        git_status: "{{ hostvars['172.31.13.150']['git_status'] }}"
  serial: 1

  tasks:
    - name: "Installing yum packages"
      yum:
        name:
          - docker 
        state: present
      register: pkg_status

    - name: "restart/enable docker service"
      when: pkg_status.changed == true
      service:
        name: docker 
        state: restarted
        enabled: yes

    - name: "Installing python docker connector module"
      pip:
        executable: pip3
        name:
          - docker
        
    - name: "Appending ec2-user to docker group"
      user:
        append: yes
        groups: docker
        name: ec2-user

    - name: Creating Docker container from latest image jinumona/devops-flask:latest"
      community.docker.docker_container:
        detach: yes
        image: jinumona/devops-flask:latest
        name: devops-flask
        published_ports: 8080:5000
        pull: yes
        restart_policy: always
        state: started 
      register: cntr_info
    
    - name: "Waiting for the health check pass of loadbalancer"
      when: cntr_info.changed == true 
      wait_for:
        sleep: 45

    - name: "Git status from the host Image-server"
      debug:
        msg: "Git version changed = {{ git_status.changed }}"

### before version change

In [ ]:
$ ansible-playbook -i hosts docker-image-git-flask-asg-rolling.yml  
[DEPRECATION WARNING]: Ansible will require Python 3.8 or newer on the controller starting with Ansible 2.12. Current version: 3.7.10 (default, Jun  3 
2021, 00:02:01) [GCC 7.3.1 20180712 (Red Hat 7.3.1-13)]. This feature will be removed from ansible-core in version 2.12. Deprecation warnings can be 
disabled by setting deprecation_warnings=False in ansible.cfg.
[WARNING]: Invalid characters were found in group names but not replaced, use -vvvv to see details

PLAY [Creating Docker Image from Git] *********************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.13.150]

TASK [Installing yum packages] ****************************************************************************************************************************
ok: [172.31.13.150]

TASK [restart/enable docker service] **********************************************************************************************************************
skipping: [172.31.13.150]

TASK [Installing python docker connector module] **********************************************************************************************************
ok: [172.31.13.150]

TASK [Appending ec2-user to docker group] *****************************************************************************************************************
ok: [172.31.13.150]

TASK [cloning git dockerfiles] ****************************************************************************************************************************
ok: [172.31.13.150]

TASK [Git fact gathering] *********************************************************************************************************************************
ok: [172.31.13.150]

TASK [Logging to DockerHub] *******************************************************************************************************************************
skipping: [172.31.13.150]

TASK [Building & Pushing Docker latest Image to Hub based on Git version 3a3ed526a9af8a3995605286fc7c3c670d745a02] ****************************************
skipping: [172.31.13.150] => (item=3a3ed526a9af8a3995605286fc7c3c670d745a02) 
skipping: [172.31.13.150] => (item=latest) 

TASK [log out from DockerHub] *****************************************************************************************************************************
skipping: [172.31.13.150]

TASK [Removing created local images] **********************************************************************************************************************
skipping: [172.31.13.150] => (item=3a3ed526a9af8a3995605286fc7c3c670d745a02) 
skipping: [172.31.13.150] => (item=latest) 

PLAY [Creating Dynamic hosts Inventry] ********************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [localhost]

TASK [Installing python aws connector modules] ************************************************************************************************************
ok: [localhost]

TASK [Fetching public ip of instances under Autoscaling Group docker-asg] *********************************************************************************
ok: [localhost]

TASK [Creating hosts inventry] ****************************************************************************************************************************
ok: [localhost] => (item={'ami_launch_index': 0, 'image_id': 'ami-08df646e18b182346', 'instance_id': 'i-062548023fff560c2', 'instance_type': 't2.micro', 'key_name': 'devops-class', 'launch_time': '2022-07-05T06:47:07+00:00', 'monitoring': {'state': 'disabled'}, 'placement': {'availability_zone': 'ap-south-1a', 'group_name': '', 'tenancy': 'default'}, 'private_dns_name': 'ip-172-31-41-215.ap-south-1.compute.internal', 'private_ip_address': '172.31.41.215', 'product_codes': [], 'public_dns_name': 'ec2-13-235-113-212.ap-south-1.compute.amazonaws.com', 'public_ip_address': '13.235.113.212', 'state': {'code': 16, 'name': 'running'}, 'state_transition_reason': '', 'subnet_id': 'subnet-0c76e66f9fa438375', 'vpc_id': 'vpc-04c6b7bd99f05c144', 'architecture': 'x86_64', 'block_device_mappings': [{'device_name': '/dev/xvda', 'ebs': {'attach_time': '2022-07-05T06:47:08+00:00', 'delete_on_termination': True, 'status': 'attached', 'volume_id': 'vol-01f45d94e32ddc159'}}], 'client_token': 'd7760734-7339-2432-c71f-45b35f2b93ff', 'ebs_optimized': False, 'ena_support': True, 'hypervisor': 'xen', 'network_interfaces': [{'association': {'ip_owner_id': 'amazon', 'public_dns_name': 'ec2-13-235-113-212.ap-south-1.compute.amazonaws.com', 'public_ip': '13.235.113.212'}, 'attachment': {'attach_time': '2022-07-05T06:47:07+00:00', 'attachment_id': 'eni-attach-00153be421f941b16', 'delete_on_termination': True, 'device_index': 0, 'status': 'attached', 'network_card_index': 0}, 'description': '', 'groups': [{'group_name': 'freedom', 'group_id': 'sg-04182bdfbfa17d831'}], 'ipv6_addresses': [], 'mac_address': '02:34:4b:fa:4f:18', 'network_interface_id': 'eni-09007e8032630b475', 'owner_id': '516234497646', 'private_dns_name': 'ip-172-31-41-215.ap-south-1.compute.internal', 'private_ip_address': '172.31.41.215', 'private_ip_addresses': [{'association': {'ip_owner_id': 'amazon', 'public_dns_name': 'ec2-13-235-113-212.ap-south-1.compute.amazonaws.com', 'public_ip': '13.235.113.212'}, 'primary': True, 'private_dns_name': 'ip-172-31-41-215.ap-south-1.compute.internal', 'private_ip_address': '172.31.41.215'}], 'source_dest_check': True, 'status': 'in-use', 'subnet_id': 'subnet-0c76e66f9fa438375', 'vpc_id': 'vpc-04c6b7bd99f05c144', 'interface_type': 'interface'}], 'root_device_name': '/dev/xvda', 'root_device_type': 'ebs', 'security_groups': [{'group_name': 'freedom', 'group_id': 'sg-04182bdfbfa17d831'}], 'source_dest_check': True, 'tags': {'Name': 'docker-asg', 'aws:autoscaling:groupName': 'docker-asg'}, 'virtualization_type': 'hvm', 'cpu_options': {'core_count': 1, 'threads_per_core': 1}, 'capacity_reservation_specification': {'capacity_reservation_preference': 'open'}, 'hibernation_options': {'configured': False}, 'metadata_options': {'state': 'applied', 'http_tokens': 'optional', 'http_put_response_hop_limit': 1, 'http_endpoint': 'enabled', 'http_protocol_ipv6': 'disabled', 'instance_metadata_tags': 'disabled'}, 'enclave_options': {'enabled': False}, 'platform_details': 'Linux/UNIX', 'usage_operation': 'RunInstances', 'usage_operation_update_time': '2022-07-05T06:47:07+00:00', 'private_dns_name_options': {'hostname_type': 'ip-name', 'enable_resource_name_dns_a_record': False, 'enable_resource_name_dns_aaaa_record': False}, 'maintenance_options': {'auto_recovery': 'default'}})
ok: [localhost] => (item={'ami_launch_index': 1, 'image_id': 'ami-08df646e18b182346', 'instance_id': 'i-06439748527e5b2f9', 'instance_type': 't2.micro', 'key_name': 'devops-class', 'launch_time': '2022-07-05T06:47:07+00:00', 'monitoring': {'state': 'disabled'}, 'placement': {'availability_zone': 'ap-south-1a', 'group_name': '', 'tenancy': 'default'}, 'private_dns_name': 'ip-172-31-37-57.ap-south-1.compute.internal', 'private_ip_address': '172.31.37.57', 'product_codes': [], 'public_dns_name': 'ec2-13-127-67-17.ap-south-1.compute.amazonaws.com', 'public_ip_address': '13.127.67.17', 'state': {'code': 16, 'name': 'running'}, 'state_transition_reason': '', 'subnet_id': 'subnet-0c76e66f9fa438375', 'vpc_id': 'vpc-04c6b7bd99f05c144', 'architecture': 'x86_64', 'block_device_mappings': [{'device_name': '/dev/xvda', 'ebs': {'attach_time': '2022-07-05T06:47:08+00:00', 'delete_on_termination': True, 'status': 'attached', 'volume_id': 'vol-057c0d0783574ee13'}}], 'client_token': 'd7760734-7339-2432-c71f-45b35f2b93ff', 'ebs_optimized': False, 'ena_support': True, 'hypervisor': 'xen', 'network_interfaces': [{'association': {'ip_owner_id': 'amazon', 'public_dns_name': 'ec2-13-127-67-17.ap-south-1.compute.amazonaws.com', 'public_ip': '13.127.67.17'}, 'attachment': {'attach_time': '2022-07-05T06:47:07+00:00', 'attachment_id': 'eni-attach-02245afdd5ad1e641', 'delete_on_termination': True, 'device_index': 0, 'status': 'attached', 'network_card_index': 0}, 'description': '', 'groups': [{'group_name': 'freedom', 'group_id': 'sg-04182bdfbfa17d831'}], 'ipv6_addresses': [], 'mac_address': '02:a4:e0:02:ec:66', 'network_interface_id': 'eni-038418535775e01ee', 'owner_id': '516234497646', 'private_dns_name': 'ip-172-31-37-57.ap-south-1.compute.internal', 'private_ip_address': '172.31.37.57', 'private_ip_addresses': [{'association': {'ip_owner_id': 'amazon', 'public_dns_name': 'ec2-13-127-67-17.ap-south-1.compute.amazonaws.com', 'public_ip': '13.127.67.17'}, 'primary': True, 'private_dns_name': 'ip-172-31-37-57.ap-south-1.compute.internal', 'private_ip_address': '172.31.37.57'}], 'source_dest_check': True, 'status': 'in-use', 'subnet_id': 'subnet-0c76e66f9fa438375', 'vpc_id': 'vpc-04c6b7bd99f05c144', 'interface_type': 'interface'}], 'root_device_name': '/dev/xvda', 'root_device_type': 'ebs', 'security_groups': [{'group_name': 'freedom', 'group_id': 'sg-04182bdfbfa17d831'}], 'source_dest_check': True, 'tags': {'Name': 'docker-asg', 'aws:autoscaling:groupName': 'docker-asg'}, 'virtualization_type': 'hvm', 'cpu_options': {'core_count': 1, 'threads_per_core': 1}, 'capacity_reservation_specification': {'capacity_reservation_preference': 'open'}, 'hibernation_options': {'configured': False}, 'metadata_options': {'state': 'applied', 'http_tokens': 'optional', 'http_put_response_hop_limit': 1, 'http_endpoint': 'enabled', 'http_protocol_ipv6': 'disabled', 'instance_metadata_tags': 'disabled'}, 'enclave_options': {'enabled': False}, 'platform_details': 'Linux/UNIX', 'usage_operation': 'RunInstances', 'usage_operation_update_time': '2022-07-05T06:47:07+00:00', 'private_dns_name_options': {'hostname_type': 'ip-name', 'enable_resource_name_dns_a_record': False, 'enable_resource_name_dns_aaaa_record': False}, 'maintenance_options': {'auto_recovery': 'default'}})
ok: [localhost] => (item={'ami_launch_index': 0, 'image_id': 'ami-08df646e18b182346', 'instance_id': 'i-0b25c5759adefbdee', 'instance_type': 't2.micro', 'key_name': 'devops-class', 'launch_time': '2022-07-05T06:47:07+00:00', 'monitoring': {'state': 'disabled'}, 'placement': {'availability_zone': 'ap-south-1b', 'group_name': '', 'tenancy': 'default'}, 'private_dns_name': 'ip-172-31-1-64.ap-south-1.compute.internal', 'private_ip_address': '172.31.1.64', 'product_codes': [], 'public_dns_name': 'ec2-13-233-59-154.ap-south-1.compute.amazonaws.com', 'public_ip_address': '13.233.59.154', 'state': {'code': 16, 'name': 'running'}, 'state_transition_reason': '', 'subnet_id': 'subnet-0513e0192526a89ad', 'vpc_id': 'vpc-04c6b7bd99f05c144', 'architecture': 'x86_64', 'block_device_mappings': [{'device_name': '/dev/xvda', 'ebs': {'attach_time': '2022-07-05T06:47:08+00:00', 'delete_on_termination': True, 'status': 'attached', 'volume_id': 'vol-04e498e37c06571c7'}}], 'client_token': '20560734-7338-339b-e42b-f467af48e083', 'ebs_optimized': False, 'ena_support': True, 'hypervisor': 'xen', 'network_interfaces': [{'association': {'ip_owner_id': 'amazon', 'public_dns_name': 'ec2-13-233-59-154.ap-south-1.compute.amazonaws.com', 'public_ip': '13.233.59.154'}, 'attachment': {'attach_time': '2022-07-05T06:47:07+00:00', 'attachment_id': 'eni-attach-06503fc1b755a6060', 'delete_on_termination': True, 'device_index': 0, 'status': 'attached', 'network_card_index': 0}, 'description': '', 'groups': [{'group_name': 'freedom', 'group_id': 'sg-04182bdfbfa17d831'}], 'ipv6_addresses': [], 'mac_address': '0a:e9:02:e0:2b:58', 'network_interface_id': 'eni-08e8859647479f9dc', 'owner_id': '516234497646', 'private_dns_name': 'ip-172-31-1-64.ap-south-1.compute.internal', 'private_ip_address': '172.31.1.64', 'private_ip_addresses': [{'association': {'ip_owner_id': 'amazon', 'public_dns_name': 'ec2-13-233-59-154.ap-south-1.compute.amazonaws.com', 'public_ip': '13.233.59.154'}, 'primary': True, 'private_dns_name': 'ip-172-31-1-64.ap-south-1.compute.internal', 'private_ip_address': '172.31.1.64'}], 'source_dest_check': True, 'status': 'in-use', 'subnet_id': 'subnet-0513e0192526a89ad', 'vpc_id': 'vpc-04c6b7bd99f05c144', 'interface_type': 'interface'}], 'root_device_name': '/dev/xvda', 'root_device_type': 'ebs', 'security_groups': [{'group_name': 'freedom', 'group_id': 'sg-04182bdfbfa17d831'}], 'source_dest_check': True, 'tags': {'aws:autoscaling:groupName': 'docker-asg', 'Name': 'docker-asg'}, 'virtualization_type': 'hvm', 'cpu_options': {'core_count': 1, 'threads_per_core': 1}, 'capacity_reservation_specification': {'capacity_reservation_preference': 'open'}, 'hibernation_options': {'configured': False}, 'metadata_options': {'state': 'applied', 'http_tokens': 'optional', 'http_put_response_hop_limit': 1, 'http_endpoint': 'enabled', 'http_protocol_ipv6': 'disabled', 'instance_metadata_tags': 'disabled'}, 'enclave_options': {'enabled': False}, 'platform_details': 'Linux/UNIX', 'usage_operation': 'RunInstances', 'usage_operation_update_time': '2022-07-05T06:47:07+00:00', 'private_dns_name_options': {'hostname_type': 'ip-name', 'enable_resource_name_dns_a_record': False, 'enable_resource_name_dns_aaaa_record': False}, 'maintenance_options': {'auto_recovery': 'default'}})

PLAY [Creating docker containers from latest Image to the backends of ASG] ********************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [13.235.113.212]

TASK [Installing yum packages] ****************************************************************************************************************************
ok: [13.235.113.212]

TASK [restart/enable docker service] **********************************************************************************************************************
skipping: [13.235.113.212]

TASK [Installing python docker connector module] **********************************************************************************************************
ok: [13.235.113.212]

TASK [Appending ec2-user to docker group] *****************************************************************************************************************
ok: [13.235.113.212]

TASK [Creating Docker container from latest image jinumona/devops-flask:latest"] **************************************************************************
ok: [13.235.113.212]

TASK [Waiting for the health check pass of loadbalancer] **************************************************************************************************
skipping: [13.235.113.212]

TASK [Git status from the host Image-server] **************************************************************************************************************
ok: [13.235.113.212] => {
    "msg": "Git version changed = False"
}

PLAY [Creating docker containers from latest Image to the backends of ASG] ********************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [13.127.67.17]

TASK [Installing yum packages] ****************************************************************************************************************************
ok: [13.127.67.17]

TASK [restart/enable docker service] **********************************************************************************************************************
skipping: [13.127.67.17]

TASK [Installing python docker connector module] **********************************************************************************************************
ok: [13.127.67.17]

TASK [Appending ec2-user to docker group] *****************************************************************************************************************
ok: [13.127.67.17]

TASK [Creating Docker container from latest image jinumona/devops-flask:latest"] **************************************************************************
ok: [13.127.67.17]

TASK [Waiting for the health check pass of loadbalancer] **************************************************************************************************
skipping: [13.127.67.17]

TASK [Git status from the host Image-server] **************************************************************************************************************
ok: [13.127.67.17] => {
    "msg": "Git version changed = False"
}

PLAY [Creating docker containers from latest Image to the backends of ASG] ********************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [13.233.59.154]

TASK [Installing yum packages] ****************************************************************************************************************************
ok: [13.233.59.154]

TASK [restart/enable docker service] **********************************************************************************************************************
skipping: [13.233.59.154]

TASK [Installing python docker connector module] **********************************************************************************************************
ok: [13.233.59.154]

TASK [Appending ec2-user to docker group] *****************************************************************************************************************
ok: [13.233.59.154]

TASK [Creating Docker container from latest image jinumona/devops-flask:latest"] **************************************************************************
ok: [13.233.59.154]

TASK [Waiting for the health check pass of loadbalancer] **************************************************************************************************
skipping: [13.233.59.154]

TASK [Git status from the host Image-server] **************************************************************************************************************
ok: [13.233.59.154] => {
    "msg": "Git version changed = False"
}

PLAY RECAP ************************************************************************************************************************************************
13.127.67.17               : ok=6    changed=0    unreachable=0    failed=0    skipped=2    rescued=0    ignored=0   
13.233.59.154              : ok=6    changed=0    unreachable=0    failed=0    skipped=2    rescued=0    ignored=0   
13.235.113.212             : ok=6    changed=0    unreachable=0    failed=0    skipped=2    rescued=0    ignored=0   
172.31.13.150              : ok=6    changed=0    unreachable=0    failed=0    skipped=5    rescued=0    ignored=0   
localhost                  : ok=4    changed=0    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0 

In [ ]:
# docker image ls
REPOSITORY              TAG       IMAGE ID       CREATED       SIZE
jinumona/devops-flask   latest    12daf034ec93   3 hours ago   64.3MB
[root@ip-172-31-41-215 ec2-user]# docker container ls -a
CONTAINER ID   IMAGE                          COMMAND            CREATED             STATUS             PORTS                    NAMES
ee83c7b40c1c   jinumona/devops-flask:latest   "python3 app.py"   About an hour ago   Up About an hour   0.0.0.0:8080->5000/tcp   devops-flask


### After version change

In [ ]:
]# docker image ls
REPOSITORY              TAG       IMAGE ID       CREATED         SIZE
jinumona/devops-flask   latest    0bd53c82a3c9   3 minutes ago   64.3MB
jinumona/devops-flask   <none>    12daf034ec93   3 hours ago     64.3MB
[root@ip-172-31-41-215 ec2-user]# docker container ls -a
CONTAINER ID   IMAGE                          COMMAND            CREATED         STATUS         PORTS                    NAMES
1707b1f46937   jinumona/devops-flask:latest   "python3 app.py"   2 minutes ago   Up 2 minutes   0.0.0.0:8080->5000/tcp   devops-flask
        

### keeping same version

In [ ]:
$ ansible-playbook -i hosts docker-image-git-flask-asg-rolling.yml

In [ ]:
$ ansible-playbook -i hosts docker-image-git-flask-asg-rolling.yml  
[DEPRECATION WARNING]: Ansible will require Python 3.8 or newer on the controller starting with Ansible 2.12. Current version: 3.7.10 (default, Jun  3 
2021, 00:02:01) [GCC 7.3.1 20180712 (Red Hat 7.3.1-13)]. This feature will be removed from ansible-core in version 2.12. Deprecation warnings can be 
disabled by setting deprecation_warnings=False in ansible.cfg.
[WARNING]: Invalid characters were found in group names but not replaced, use -vvvv to see details

PLAY [Creating Docker Image from Git] *********************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.13.150]

TASK [Installing yum packages] ****************************************************************************************************************************
ok: [172.31.13.150]

TASK [restart/enable docker service] **********************************************************************************************************************
skipping: [172.31.13.150]

TASK [Installing python docker connector module] **********************************************************************************************************
ok: [172.31.13.150]

TASK [Appending ec2-user to docker group] *****************************************************************************************************************
ok: [172.31.13.150]

TASK [cloning git dockerfiles] ****************************************************************************************************************************
ok: [172.31.13.150]

TASK [Git fact gathering] *********************************************************************************************************************************
ok: [172.31.13.150]

TASK [Logging to DockerHub] *******************************************************************************************************************************
skipping: [172.31.13.150]

TASK [Building & Pushing Docker latest Image to Hub based on Git version 11be5ec83ed96c89a43a0634c13eecb9b565b4ec] ****************************************
skipping: [172.31.13.150] => (item=11be5ec83ed96c89a43a0634c13eecb9b565b4ec) 
skipping: [172.31.13.150] => (item=latest) 

TASK [log out from DockerHub] *****************************************************************************************************************************
skipping: [172.31.13.150]

TASK [Removing created local images] **********************************************************************************************************************
skipping: [172.31.13.150] => (item=11be5ec83ed96c89a43a0634c13eecb9b565b4ec) 
skipping: [172.31.13.150] => (item=latest) 

PLAY [Creating Dynamic hosts Inventry] ********************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [localhost]

TASK [Installing python aws connector modules] ************************************************************************************************************
ok: [localhost]

TASK [Fetching public ip of instances under Autoscaling Group docker-asg] *********************************************************************************
ok: [localhost]

TASK [Creating hosts inventry] ****************************************************************************************************************************
ok: [localhost] => (item={'ami_launch_index': 0, 'image_id': 'ami-08df646e18b182346', 'instance_id': 'i-062548023fff560c2', 'instance_type': 't2.micro', 'key_name': 'devops-class', 'launch_time': '2022-07-05T06:47:07+00:00', 'monitoring': {'state': 'disabled'}, 'placement': {'availability_zone': 'ap-south-1a', 'group_name': '', 'tenancy': 'default'}, 'private_dns_name': 'ip-172-31-41-215.ap-south-1.compute.internal', 'private_ip_address': '172.31.41.215', 'product_codes': [], 'public_dns_name': 'ec2-13-235-113-212.ap-south-1.compute.amazonaws.com', 'public_ip_address': '13.235.113.212', 'state': {'code': 16, 'name': 'running'}, 'state_transition_reason': '', 'subnet_id': 'subnet-0c76e66f9fa438375', 'vpc_id': 'vpc-04c6b7bd99f05c144', 'architecture': 'x86_64', 'block_device_mappings': [{'device_name': '/dev/xvda', 'ebs': {'attach_time': '2022-07-05T06:47:08+00:00', 'delete_on_termination': True, 'status': 'attached', 'volume_id': 'vol-01f45d94e32ddc159'}}], 'client_token': 'd7760734-7339-2432-c71f-45b35f2b93ff', 'ebs_optimized': False, 'ena_support': True, 'hypervisor': 'xen', 'network_interfaces': [{'association': {'ip_owner_id': 'amazon', 'public_dns_name': 'ec2-13-235-113-212.ap-south-1.compute.amazonaws.com', 'public_ip': '13.235.113.212'}, 'attachment': {'attach_time': '2022-07-05T06:47:07+00:00', 'attachment_id': 'eni-attach-00153be421f941b16', 'delete_on_termination': True, 'device_index': 0, 'status': 'attached', 'network_card_index': 0}, 'description': '', 'groups': [{'group_name': 'freedom', 'group_id': 'sg-04182bdfbfa17d831'}], 'ipv6_addresses': [], 'mac_address': '02:34:4b:fa:4f:18', 'network_interface_id': 'eni-09007e8032630b475', 'owner_id': '516234497646', 'private_dns_name': 'ip-172-31-41-215.ap-south-1.compute.internal', 'private_ip_address': '172.31.41.215', 'private_ip_addresses': [{'association': {'ip_owner_id': 'amazon', 'public_dns_name': 'ec2-13-235-113-212.ap-south-1.compute.amazonaws.com', 'public_ip': '13.235.113.212'}, 'primary': True, 'private_dns_name': 'ip-172-31-41-215.ap-south-1.compute.internal', 'private_ip_address': '172.31.41.215'}], 'source_dest_check': True, 'status': 'in-use', 'subnet_id': 'subnet-0c76e66f9fa438375', 'vpc_id': 'vpc-04c6b7bd99f05c144', 'interface_type': 'interface'}], 'root_device_name': '/dev/xvda', 'root_device_type': 'ebs', 'security_groups': [{'group_name': 'freedom', 'group_id': 'sg-04182bdfbfa17d831'}], 'source_dest_check': True, 'tags': {'Name': 'docker-asg', 'aws:autoscaling:groupName': 'docker-asg'}, 'virtualization_type': 'hvm', 'cpu_options': {'core_count': 1, 'threads_per_core': 1}, 'capacity_reservation_specification': {'capacity_reservation_preference': 'open'}, 'hibernation_options': {'configured': False}, 'metadata_options': {'state': 'applied', 'http_tokens': 'optional', 'http_put_response_hop_limit': 1, 'http_endpoint': 'enabled', 'http_protocol_ipv6': 'disabled', 'instance_metadata_tags': 'disabled'}, 'enclave_options': {'enabled': False}, 'platform_details': 'Linux/UNIX', 'usage_operation': 'RunInstances', 'usage_operation_update_time': '2022-07-05T06:47:07+00:00', 'private_dns_name_options': {'hostname_type': 'ip-name', 'enable_resource_name_dns_a_record': False, 'enable_resource_name_dns_aaaa_record': False}, 'maintenance_options': {'auto_recovery': 'default'}})
ok: [localhost] => (item={'ami_launch_index': 1, 'image_id': 'ami-08df646e18b182346', 'instance_id': 'i-06439748527e5b2f9', 'instance_type': 't2.micro', 'key_name': 'devops-class', 'launch_time': '2022-07-05T06:47:07+00:00', 'monitoring': {'state': 'disabled'}, 'placement': {'availability_zone': 'ap-south-1a', 'group_name': '', 'tenancy': 'default'}, 'private_dns_name': 'ip-172-31-37-57.ap-south-1.compute.internal', 'private_ip_address': '172.31.37.57', 'product_codes': [], 'public_dns_name': 'ec2-13-127-67-17.ap-south-1.compute.amazonaws.com', 'public_ip_address': '13.127.67.17', 'state': {'code': 16, 'name': 'running'}, 'state_transition_reason': '', 'subnet_id': 'subnet-0c76e66f9fa438375', 'vpc_id': 'vpc-04c6b7bd99f05c144', 'architecture': 'x86_64', 'block_device_mappings': [{'device_name': '/dev/xvda', 'ebs': {'attach_time': '2022-07-05T06:47:08+00:00', 'delete_on_termination': True, 'status': 'attached', 'volume_id': 'vol-057c0d0783574ee13'}}], 'client_token': 'd7760734-7339-2432-c71f-45b35f2b93ff', 'ebs_optimized': False, 'ena_support': True, 'hypervisor': 'xen', 'network_interfaces': [{'association': {'ip_owner_id': 'amazon', 'public_dns_name': 'ec2-13-127-67-17.ap-south-1.compute.amazonaws.com', 'public_ip': '13.127.67.17'}, 'attachment': {'attach_time': '2022-07-05T06:47:07+00:00', 'attachment_id': 'eni-attach-02245afdd5ad1e641', 'delete_on_termination': True, 'device_index': 0, 'status': 'attached', 'network_card_index': 0}, 'description': '', 'groups': [{'group_name': 'freedom', 'group_id': 'sg-04182bdfbfa17d831'}], 'ipv6_addresses': [], 'mac_address': '02:a4:e0:02:ec:66', 'network_interface_id': 'eni-038418535775e01ee', 'owner_id': '516234497646', 'private_dns_name': 'ip-172-31-37-57.ap-south-1.compute.internal', 'private_ip_address': '172.31.37.57', 'private_ip_addresses': [{'association': {'ip_owner_id': 'amazon', 'public_dns_name': 'ec2-13-127-67-17.ap-south-1.compute.amazonaws.com', 'public_ip': '13.127.67.17'}, 'primary': True, 'private_dns_name': 'ip-172-31-37-57.ap-south-1.compute.internal', 'private_ip_address': '172.31.37.57'}], 'source_dest_check': True, 'status': 'in-use', 'subnet_id': 'subnet-0c76e66f9fa438375', 'vpc_id': 'vpc-04c6b7bd99f05c144', 'interface_type': 'interface'}], 'root_device_name': '/dev/xvda', 'root_device_type': 'ebs', 'security_groups': [{'group_name': 'freedom', 'group_id': 'sg-04182bdfbfa17d831'}], 'source_dest_check': True, 'tags': {'Name': 'docker-asg', 'aws:autoscaling:groupName': 'docker-asg'}, 'virtualization_type': 'hvm', 'cpu_options': {'core_count': 1, 'threads_per_core': 1}, 'capacity_reservation_specification': {'capacity_reservation_preference': 'open'}, 'hibernation_options': {'configured': False}, 'metadata_options': {'state': 'applied', 'http_tokens': 'optional', 'http_put_response_hop_limit': 1, 'http_endpoint': 'enabled', 'http_protocol_ipv6': 'disabled', 'instance_metadata_tags': 'disabled'}, 'enclave_options': {'enabled': False}, 'platform_details': 'Linux/UNIX', 'usage_operation': 'RunInstances', 'usage_operation_update_time': '2022-07-05T06:47:07+00:00', 'private_dns_name_options': {'hostname_type': 'ip-name', 'enable_resource_name_dns_a_record': False, 'enable_resource_name_dns_aaaa_record': False}, 'maintenance_options': {'auto_recovery': 'default'}})
ok: [localhost] => (item={'ami_launch_index': 0, 'image_id': 'ami-08df646e18b182346', 'instance_id': 'i-0b25c5759adefbdee', 'instance_type': 't2.micro', 'key_name': 'devops-class', 'launch_time': '2022-07-05T06:47:07+00:00', 'monitoring': {'state': 'disabled'}, 'placement': {'availability_zone': 'ap-south-1b', 'group_name': '', 'tenancy': 'default'}, 'private_dns_name': 'ip-172-31-1-64.ap-south-1.compute.internal', 'private_ip_address': '172.31.1.64', 'product_codes': [], 'public_dns_name': 'ec2-13-233-59-154.ap-south-1.compute.amazonaws.com', 'public_ip_address': '13.233.59.154', 'state': {'code': 16, 'name': 'running'}, 'state_transition_reason': '', 'subnet_id': 'subnet-0513e0192526a89ad', 'vpc_id': 'vpc-04c6b7bd99f05c144', 'architecture': 'x86_64', 'block_device_mappings': [{'device_name': '/dev/xvda', 'ebs': {'attach_time': '2022-07-05T06:47:08+00:00', 'delete_on_termination': True, 'status': 'attached', 'volume_id': 'vol-04e498e37c06571c7'}}], 'client_token': '20560734-7338-339b-e42b-f467af48e083', 'ebs_optimized': False, 'ena_support': True, 'hypervisor': 'xen', 'network_interfaces': [{'association': {'ip_owner_id': 'amazon', 'public_dns_name': 'ec2-13-233-59-154.ap-south-1.compute.amazonaws.com', 'public_ip': '13.233.59.154'}, 'attachment': {'attach_time': '2022-07-05T06:47:07+00:00', 'attachment_id': 'eni-attach-06503fc1b755a6060', 'delete_on_termination': True, 'device_index': 0, 'status': 'attached', 'network_card_index': 0}, 'description': '', 'groups': [{'group_name': 'freedom', 'group_id': 'sg-04182bdfbfa17d831'}], 'ipv6_addresses': [], 'mac_address': '0a:e9:02:e0:2b:58', 'network_interface_id': 'eni-08e8859647479f9dc', 'owner_id': '516234497646', 'private_dns_name': 'ip-172-31-1-64.ap-south-1.compute.internal', 'private_ip_address': '172.31.1.64', 'private_ip_addresses': [{'association': {'ip_owner_id': 'amazon', 'public_dns_name': 'ec2-13-233-59-154.ap-south-1.compute.amazonaws.com', 'public_ip': '13.233.59.154'}, 'primary': True, 'private_dns_name': 'ip-172-31-1-64.ap-south-1.compute.internal', 'private_ip_address': '172.31.1.64'}], 'source_dest_check': True, 'status': 'in-use', 'subnet_id': 'subnet-0513e0192526a89ad', 'vpc_id': 'vpc-04c6b7bd99f05c144', 'interface_type': 'interface'}], 'root_device_name': '/dev/xvda', 'root_device_type': 'ebs', 'security_groups': [{'group_name': 'freedom', 'group_id': 'sg-04182bdfbfa17d831'}], 'source_dest_check': True, 'tags': {'aws:autoscaling:groupName': 'docker-asg', 'Name': 'docker-asg'}, 'virtualization_type': 'hvm', 'cpu_options': {'core_count': 1, 'threads_per_core': 1}, 'capacity_reservation_specification': {'capacity_reservation_preference': 'open'}, 'hibernation_options': {'configured': False}, 'metadata_options': {'state': 'applied', 'http_tokens': 'optional', 'http_put_response_hop_limit': 1, 'http_endpoint': 'enabled', 'http_protocol_ipv6': 'disabled', 'instance_metadata_tags': 'disabled'}, 'enclave_options': {'enabled': False}, 'platform_details': 'Linux/UNIX', 'usage_operation': 'RunInstances', 'usage_operation_update_time': '2022-07-05T06:47:07+00:00', 'private_dns_name_options': {'hostname_type': 'ip-name', 'enable_resource_name_dns_a_record': False, 'enable_resource_name_dns_aaaa_record': False}, 'maintenance_options': {'auto_recovery': 'default'}})

PLAY [Creating docker containers from latest Image to the backends of ASG] ********************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [13.235.113.212]

TASK [Installing yum packages] ****************************************************************************************************************************
ok: [13.235.113.212]

TASK [restart/enable docker service] **********************************************************************************************************************
skipping: [13.235.113.212]

TASK [Installing python docker connector module] **********************************************************************************************************
ok: [13.235.113.212]

TASK [Appending ec2-user to docker group] *****************************************************************************************************************
ok: [13.235.113.212]

TASK [Creating Docker container from latest image jinumona/devops-flask:latest"] **************************************************************************
ok: [13.235.113.212]

TASK [Waiting for the health check pass of loadbalancer] **************************************************************************************************
skipping: [13.235.113.212]

TASK [Git status from the host Image-server] **************************************************************************************************************
ok: [13.235.113.212] => {
    "msg": "Git version changed = False"
}

PLAY [Creating docker containers from latest Image to the backends of ASG] ********************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [13.127.67.17]

TASK [Installing yum packages] ****************************************************************************************************************************
ok: [13.127.67.17]

TASK [restart/enable docker service] **********************************************************************************************************************
skipping: [13.127.67.17]

TASK [Installing python docker connector module] **********************************************************************************************************
ok: [13.127.67.17]

TASK [Appending ec2-user to docker group] *****************************************************************************************************************
ok: [13.127.67.17]

TASK [Creating Docker container from latest image jinumona/devops-flask:latest"] **************************************************************************
ok: [13.127.67.17]

TASK [Waiting for the health check pass of loadbalancer] **************************************************************************************************
skipping: [13.127.67.17]

TASK [Git status from the host Image-server] **************************************************************************************************************
ok: [13.127.67.17] => {
    "msg": "Git version changed = False"
}

PLAY [Creating docker containers from latest Image to the backends of ASG] ********************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [13.233.59.154]

TASK [Installing yum packages] ****************************************************************************************************************************
ok: [13.233.59.154]

TASK [restart/enable docker service] **********************************************************************************************************************
skipping: [13.233.59.154]

TASK [Installing python docker connector module] **********************************************************************************************************
ok: [13.233.59.154]

TASK [Appending ec2-user to docker group] *****************************************************************************************************************
ok: [13.233.59.154]

TASK [Creating Docker container from latest image jinumona/devops-flask:latest"] **************************************************************************
ok: [13.233.59.154]

TASK [Waiting for the health check pass of loadbalancer] **************************************************************************************************
skipping: [13.233.59.154]

TASK [Git status from the host Image-server] **************************************************************************************************************
ok: [13.233.59.154] => {
    "msg": "Git version changed = False"
}

PLAY RECAP ************************************************************************************************************************************************
13.127.67.17               : ok=6    changed=0    unreachable=0    failed=0    skipped=2    rescued=0    ignored=0   
13.233.59.154              : ok=6    changed=0    unreachable=0    failed=0    skipped=2    rescued=0    ignored=0   
13.235.113.212             : ok=6    changed=0    unreachable=0    failed=0    skipped=2    rescued=0    ignored=0   
172.31.13.150              : ok=6    changed=0    unreachable=0    failed=0    skipped=5    rescued=0    ignored=0   
localhost                  : ok=4    changed=0    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0 